In [5]:
import numpy as np
import pandas as pd
import sys
sys.path.append(r'C:\Users\JOSE\Desktop\Trabajo\BX')
import funciones as fn
import re

In [3]:
df = pd.read_excel(r'C:\Users\JOSE\Desktop\Trabajo\BX\Supervisado\Base_semanal_pegada.xlsx')
df

,Unnamed: 0,Date,Snippet,Domain,Sentiment,City_Code,Account_Type,Author,City,Gender,...,Categoría_Barometro_v4_-_Denuncia_Xenofobia,Categoría_Barometro_v4_-_Xenofobia,Categorías_prueba_-_Denuncia,Categorías_prueba_-_Xenofobia,Categorías_prueba_-_denuncia_xenofobia_cl,Categorías_prueba_-_denuncia_xenofobia_col,xenofobia_e_Integración_-_Integración,xenofobia_e_Integración_-_Xenofobia,xenofobia_e_Integración_-_not_xenofobia,Clean_text
0,0,2020-01-08 23:59:57.0,Justamente le acabe el guacamole picante al ve...,twitter.com,neutral,COL.Bogota.Bogota,individual,Hector_Jk7,Bogota,male,...,NaN,NaN,NaN,NaN,X,X,NaN,X,X,Justamente le acabe el guacamole picante al ve...
1,2,2020-01-08 23:59:20.0,...1141486160931430407] El seleccionado catarí...,vanguardia.com,neutral,NaN,NaN,NaN,NaN,unknown,...,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,"..1141486160931430407] El seleccionado catarí,..."
2,4,2020-01-08 23:58:10.0,@NTN24ve EL QUE NO SABE CON QUIEN SE METE ES L...,twitter.com,positive,COL.Antioquia.Medellin,individual,LeilaLozano3,Medellin,female,...,NaN,NaN,NaN,NaN,X,X,NaN,X,X,@NTN24ve EL QUE NO SABE CON QUIEN SE METE ES L...
3,5,2020-01-08 23:55:42.0,@NoticiasCaracol Los venezolanos siempre esper...,twitter.com,negative,COL.Antioquia.Medellin,individual,0rwell07,Medellin,male,...,NaN,NaN,NaN,NaN,X,X,NaN,NaN,X,@NoticiasCaracol Los venezolanos siempre esper...
4,7,2020-01-08 23:54:14.0,Los 6 puntos q trata la cartica denota su prox...,twitter.com,neutral,COL.Bogota.Bogota,individual,PartidoRepubl14,Bogota,unknown,...,NaN,NaN,NaN,NaN,X,X,NaN,X,X,Los 6 puntos q trata la cartica denota su prox...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350574,46995,2021-03-08 00:11:19.0,@yoligrilla @elespectador Y este gobierno colo...,twitter.com,neutral,NaN,individual,RamonEdo,NaN,unknown,...,NaN,NaN,NaN,NaN,NaN,X,NaN,X,NaN,@yoligrilla @elespectador Y este gobierno colo...
350575,46996,2021-03-08 00:10:35.0,"..., y de taquito se la sirvió a Yesus Cabrera...",primertiempo.co,negative,NaN,NaN,NaN,NaN,unknown,...,NaN,NaN,NaN,NaN,X,X,NaN,NaN,NaN,".., y de taquito se la sirvió a Yesus Cabrera...."
350576,47000,2021-03-08 00:07:01.0,"Eduardo Sosa, José David Barragán y José Luis ...",twitter.com,positive,COL.Bogota.Bogota,individual,riosgonzalezp,Bogota,male,...,NaN,NaN,NaN,NaN,X,NaN,NaN,NaN,NaN,"Eduardo Sosa, José David Barragán y José Luis ..."
350577,47001,2021-03-08 00:05:02.0,La venezolana Jenny Camacho es la mente maestr...,twitter.com,neutral,COL.Bogota.Bogota,individual,dinersrevista,Bogota,unknown,...,NaN,NaN,NaN,NaN,X,X,NaN,NaN,X,La venezolana Jenny Camacho es la mente maestr...


In [4]:
for col in df.columns.to_list():
    print(col)

Unnamed: 0
Date
Snippet
Domain
Sentiment
City_Code
Account_Type
Author
City
Gender
Hashtags
Impact
Impressions
Interest
Mentioned_Authors
Professions
Thread_Author
Thread_Entry_Type
Total_Monthly_Visitors
Twitter_Followers
Twitter_Following
Twitter_Reply_Count
Twitter_Retweets
Twitter_Tweets
Twitter_Verified
Reach_(new)
Categoría_Barometro_v3_-_Seguridad_
Categoría_Barometro_v4_-_Denuncia_Xenofobia
Categoría_Barometro_v4_-_Xenofobia
Categorías_prueba_-_Denuncia
Categorías_prueba_-_Xenofobia
Categorías_prueba_-_denuncia_xenofobia_cl
Categorías_prueba_-_denuncia_xenofobia_col
xenofobia_e_Integración_-_Integración
xenofobia_e_Integración_-_Xenofobia
xenofobia_e_Integración_-_not_xenofobia
Clean_text


In [7]:
print(df.loc[3])

Unnamed: 0                                                                                     5
Date                                                                       2020-01-08 23:55:42.0
Snippet                                        @NoticiasCaracol Los venezolanos siempre esper...
Domain                                                                               twitter.com
Sentiment                                                                               negative
City_Code                                                                 COL.Antioquia.Medellin
Account_Type                                                                          individual
Author                                                                                  0rwell07
City                                                                                    Medellin
Gender                                                                                      male
Hashtags                      

In [8]:
df['Thread_Entry_Type'].unique()

array(['post', 'reply', 'share'], dtype=object)

In [24]:
# Nos vamos a quedar solo con medios, policía y figuras
# Importamos autores de medios
text = open(r"C:\Users\JOSE\Desktop\Trabajo\BX\Denuncia Xenofobia\medios.txt","r")
medios = list()
for line in text:
    line = line.rsplit("OR")
    line = line[0].strip()
    #if line.find(".") == -1:
    #    line = "@" + line
    medios.append(line)
# Vamos a guardar la lista de autores. Primero policia, luego medios tradicionales y después, alternos
medios_policia = ["PoliciaBogota", "region6policia", "region7policia", "Region8Policia", "Region2Policia", "Region3Policia", "Region1Policia",
                    "Region4Policia", "Region5Policia", "PoliciaNarino", "DevalPolicia", "PoliciaMeta", "PoliciaMedellin", "PoliciaBoyaca",
                    "PoliciaMzales", "PoliciaVillavo", "PoliciaBmanga", "PoliciaCali", "PoliciaDEMAG", "PoliciaCmarca", "Policiantioquia",
                    "PoliciaCucuta", "PoliciaQuindio", "Policia_Caldas", "SucrePolicia", "PoliciaVichada", "PoliciaGuainia",
                    "PoliciaArauca", "PoliciaCasanare", "GuajiraPolicia", "PoliciaVaupes", "PoliciaChoco", "PoliciaDepuy",
                    "PoliciaUraba", "PoliciAtlantico", "PoliciaDECOR", "PoliciaNteSder", "HuilaPolicia", "PoliciaBolivar", "PoliciaCaqueta",
                    "PoliciaDeris", "PoliciaDeTolima", "PoliciaDEMAM", "PoliciaMTunja", "PoliciaMonteria", "PoliciAmazonas", "PoliciaDeCesar",
                    "PoliciaSanyProv", "PoliciaPasto", "PoliciaStander", "PoliciaStaMarta", "PoliciaPereira", "PoliciaIbague", "PoliciaPopayan", 
                    "PoliciaCtagena", "PoliciaColombia"]
#Medios alternativos
text = open(r"C:\Users\JOSE\Desktop\Trabajo\BX\Denuncia Xenofobia\alternos.txt","r")
alternos = list()
for line in text:
    line = line.rsplit("OR")
    line = line[0].strip()
    #if line.find(".") == -1:
    #    line = "@" + line
    alternos.append(line)
figuras = ['IvanDuque', 'ClaudiaLopez', 'QuinteroCalle', 'JorgeIvanOspina', 'jaimepumarejo', 'nicolasgarciab', 
           'ernestosamperp', 'LaPullaOficial', 'DanielSamperO',
                'MariaFdaCabal', 'AlvaroUribeVel', 'petrogustavo', 'GustavoBolivar', 
           'JuanManSantos', 'PalomaValenciaL', 'AlcaldiadeMed', 'Bogota', 'sergio_fajardo', 'EstradaCarlosM', 
                'seguridadmed', 'SeguridadBOG']
todos_medios = alternos + medios + medios_policia + figuras
todos_medios

['D_C_Bogota',
 'Agencia_API',
 'ArgumentosyOpi',
 'lasillavacia',
 'noticucuta',
 '2minutosco',
 'Elsonajeroweb',
 'siaradio',
 'noticierolaruta',
 'AlertaPaisa',
 'orientesetuguia',
 'acuariotv',
 'cncnoticias',
 'Informativoan',
 'MiPutumayo',
 'zonacero',
 'AtlanticoEmi',
 'primeronoticiaa1',
 'primeronoticiaa',
 'CVnoticiastv',
 'LaLibertadCo',
 'CanalTropicaltv',
 'elambito',
 'NorteSeguro2018',
 'areacucuta',
 'CucutaDenuncia',
 'WALLTHER_VE',
 'noticucuta',
 'NorteNoticiasCu',
 'EnterateCali',
 'norte_periodico',
 'canalguacari',
 'TwiterosCali',
 'YaCelacanto',
 'HablaManizales',
 'tintiando',
 'NCTNOTICIAS',
 '1anoticiass',
 'OsBoyaca',
 'aldianoticiasco',
 'LaIndependenci1',
 'Prensalibcasan',
 'elconucoacacias',
 'EIExpedienteCol',
 'hablalonoticias',
 '7NNoticias',
 'ElPregonar',
 '45SegundosCom',
 'AlPuntoCol',
 'ColmundoRadio',
 'ElVenezolanoCo',
 'ExtraColombia',
 'HSBnoticias',
 'rodrigoprensa',
 'Yariguies1027',
 'BLUSantanderes',
 'Villavoalreves',
 'Region_365',
 'E

In [69]:
# ESTE CÓDIGO ES UN BOOLEAN MASK HECHO A MANO, PUEDE TARDAR MUCHO
nonan = df[~df['Thread_Author'].isnull()]
tipos = [alternos, medios, medios_policia, figuras]
nombres = ['alternos', 'tradicionales', 'policia','figuras']
nonan['Tipo_de_autor'] = ''
i = 0
for tipo in tipos:
    nombre = nombres[i]
    i += 1
    for autor in tipo:
        nonan.loc[nonan.Thread_Author == autor, 'Tipo_de_autor'] = nombre
datos = nonan[np.logical_or.reduce([nonan['Thread_Author'].str.contains(autor) for autor in todos_medios])]
datos = datos.reset_index()
datos

<ipython-input-69-888416789af0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonan['Tipo_de_autor'] = ''
C:\Users\JOSE\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Unnamed: 0,Date,Snippet,Domain,Sentiment,City_Code,Account_Type,Author,City,...,Categoría_Barometro_v4_-_Xenofobia,Categorías_prueba_-_Denuncia,Categorías_prueba_-_Xenofobia,Categorías_prueba_-_denuncia_xenofobia_cl,Categorías_prueba_-_denuncia_xenofobia_col,xenofobia_e_Integración_-_Integración,xenofobia_e_Integración_-_Xenofobia,xenofobia_e_Integración_-_not_xenofobia,Clean_text,Tipo_de_autor
0,2,4,2020-01-08 23:58:10.0,@NTN24ve EL QUE NO SABE CON QUIEN SE METE ES L...,twitter.com,positive,COL.Antioquia.Medellin,individual,LeilaLozano3,Medellin,...,NaN,NaN,NaN,X,X,NaN,X,X,@NTN24ve EL QUE NO SABE CON QUIEN SE METE ES L...,
1,3,5,2020-01-08 23:55:42.0,@NoticiasCaracol Los venezolanos siempre esper...,twitter.com,negative,COL.Antioquia.Medellin,individual,0rwell07,Medellin,...,NaN,NaN,NaN,X,X,NaN,NaN,X,@NoticiasCaracol Los venezolanos siempre esper...,tradicionales
2,15,34,2020-01-08 23:32:58.0,@petrogustavo Yo me imagino una economía en la...,twitter.com,negative,NaN,individual,MrSwanMods,NaN,...,NaN,NaN,NaN,X,X,NaN,NaN,X,@petrogustavo Yo me imagino una economía en la...,figuras
3,36,76,2020-01-08 22:51:46.0,"@IvanDuque Menos hablabla y más acción, ahora ...",twitter.com,negative,COL.Bogota.Bogota,individual,billyjohnns,Bogota,...,NaN,NaN,NaN,X,X,NaN,X,X,"@IvanDuque Menos hablabla y más acción, ahora ...",figuras
4,108,225,2020-01-08 20:36:22.0,"@NoticiasCaracol Veneco hijo de puta, merece l...",twitter.com,negative,COL.Bogota.Bogota,individual,carlomagno2704,Bogota,...,NaN,NaN,NaN,X,X,NaN,X,X,"@NoticiasCaracol Veneco hijo de puta, merece l...",tradicionales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31110,350417,46598,2021-03-08 12:07:38.0,@petrogustavo Así eres tú un aprovechado popul...,twitter.com,negative,COL.Bogota.Bogota,individual,andirup,Bogota,...,NaN,NaN,NaN,X,NaN,X,NaN,NaN,@petrogustavo Así eres tú un aprovechado popul...,figuras
31111,350419,46604,2021-03-08 12:04:41.0,@NTN24ve Y las Vacunas para los Venezolanos? Q...,twitter.com,negative,COL.Atlantico.Barranquilla,individual,MAPY2,Barranquilla,...,NaN,NaN,NaN,X,X,NaN,NaN,X,@NTN24ve Y las Vacunas para los Venezolanos? Q...,
31112,350478,46753,2021-03-08 04:20:21.0,@Citytv Una avalancha de venecos y dos millone...,twitter.com,neutral,COL.Bogota.Bogota,individual,alexfajardo71,Bogota,...,NaN,NaN,NaN,X,X,NaN,X,X,@Citytv Una avalancha de venecos y dos millone...,tradicionales
31113,350553,46946,2021-03-08 00:56:15.0,@PublimetroCol Estas son las “ideas” nefastas ...,twitter.com,negative,NaN,individual,DiegoAl50239232,NaN,...,NaN,NaN,NaN,X,X,NaN,X,X,@PublimetroCol Estas son las “ideas” nefastas ...,tradicionales


In [70]:
vacios = datos[datos['Tipo_de_autor'] == '']
print(vacios['Thread_Author'].unique())

['NTN24ve' 'Bogota__DC' 'ActivistaBogota' 'ArribaBogota' 'BogotaTransito'
 'BogotaET' 'AlertaBogota' 'CGurisattiNTN24' 'MirandaBogota' 'elBogotano'
 'DadepBogota' 'ConcejoDeBogota' 'NCBogota' 'LaNocheNTN24'
 'BogotanoAnonimo' 'Parque93Bogota' 'BogotaInter_nal' 'lafmmedellin'
 'BogotaMiBarrio' 'JeffersonNTN24' 'VeeduriaBogota' 'BomberosBogota'
 'BogotaParticipa' 'VladoBogota' 'VzlanosBogota' 'elcolombianox19'
 'LasernaBogota' 'UnaBogotana' 'SrBogotano' 'BogotaCurioso' 'MillanBogota'
 'NovioBogota' 'USEmbassyBogota' 'YoSoyDeBogota' 'Fontibon_Bogota'
 'TerminalBogota' 'Bogota31610272' 'DonBogotano' 'MonoMarioBogota'
 'ElSolBogota' 'GuzmanBogota' 'NTN24Zoom' 'VenecoBogota' 'ArchivodeBogota'
 'lafmdeleje' '23Bogota' 'CPoderNTN24' 'MetroBogota' 'ElBogotan0'
 'BogotaLogica' 'FabianLBogota' 'EnamorateBogota' 'BogotaCruising'
 'AjiDulceBogota' 'BogotaEE' 'Bogotaensis' 'ClubPrensaNTN24' 'TropiBogota'
 'GerardoBogota' 'BogotaHumana_' 'DjxBogota' 'NayiBogota' 'BogotaseMueve'
 'UNICDBogota' 'Activo

In [71]:
coldrop = ['Categoría_Barometro_v3_-_Seguridad_' ,
'Categoría_Barometro_v4_-_Denuncia_Xenofobia',
'Categoría_Barometro_v4_-_Xenofobia',
'Categorías_prueba_-_Denuncia',
'Categorías_prueba_-_Xenofobia',
'Categorías_prueba_-_denuncia_xenofobia_cl',
'Categorías_prueba_-_denuncia_xenofobia_col',
'index','Unnamed: 0', 'xenofobia_e_Integración_-_not_xenofobia']
datos = datos.drop(coldrop, axis = 1)
datos['xenofobia_e_Integración_-_Xenofobia'] = datos['xenofobia_e_Integración_-_Xenofobia'].map({"NaN":0 , np.nan:0, 'X':1, 'x':1})
datos['xenofobia_e_Integración_-_Integración'] = datos['xenofobia_e_Integración_-_Integración'].map({"NaN":0, np.nan:0 , 'X':1, 'x':1})
datos

,Date,Snippet,Domain,Sentiment,City_Code,Account_Type,Author,City,Gender,Hashtags,...,Twitter_Following,Twitter_Reply_Count,Twitter_Retweets,Twitter_Tweets,Twitter_Verified,Reach_(new),xenofobia_e_Integración_-_Integración,xenofobia_e_Integración_-_Xenofobia,Clean_text,Tipo_de_autor
0,2020-01-08 23:58:10.0,@NTN24ve EL QUE NO SABE CON QUIEN SE METE ES L...,twitter.com,positive,COL.Antioquia.Medellin,individual,LeilaLozano3,Medellin,female,NaN,...,21,0,0,878,False,0.0,0,1,@NTN24ve EL QUE NO SABE CON QUIEN SE METE ES L...,
1,2020-01-08 23:55:42.0,@NoticiasCaracol Los venezolanos siempre esper...,twitter.com,negative,COL.Antioquia.Medellin,individual,0rwell07,Medellin,male,NaN,...,287,0,0,13374,False,0.0,0,0,@NoticiasCaracol Los venezolanos siempre esper...,tradicionales
2,2020-01-08 23:32:58.0,@petrogustavo Yo me imagino una economía en la...,twitter.com,negative,NaN,individual,MrSwanMods,NaN,unknown,NaN,...,102,0,0,1216,False,0.0,0,0,@petrogustavo Yo me imagino una economía en la...,figuras
3,2020-01-08 22:51:46.0,"@IvanDuque Menos hablabla y más acción, ahora ...",twitter.com,negative,COL.Bogota.Bogota,individual,billyjohnns,Bogota,male,NaN,...,364,0,0,115,False,0.0,0,1,"@IvanDuque Menos hablabla y más acción, ahora ...",figuras
4,2020-01-08 20:36:22.0,"@NoticiasCaracol Veneco hijo de puta, merece l...",twitter.com,negative,COL.Bogota.Bogota,individual,carlomagno2704,Bogota,male,NaN,...,4997,0,0,36459,False,3774.0,0,1,"@NoticiasCaracol Veneco hijo de puta, merece l...",tradicionales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31110,2021-03-08 12:07:38.0,@petrogustavo Así eres tú un aprovechado popul...,twitter.com,negative,COL.Bogota.Bogota,individual,andirup,Bogota,male,NaN,...,39,0,0,352,False,0.0,1,0,@petrogustavo Así eres tú un aprovechado popul...,figuras
31111,2021-03-08 12:04:41.0,@NTN24ve Y las Vacunas para los Venezolanos? Q...,twitter.com,negative,COL.Atlantico.Barranquilla,individual,MAPY2,Barranquilla,female,NaN,...,460,0,0,14770,False,0.0,0,0,@NTN24ve Y las Vacunas para los Venezolanos? Q...,
31112,2021-03-08 04:20:21.0,@Citytv Una avalancha de venecos y dos millone...,twitter.com,neutral,COL.Bogota.Bogota,individual,alexfajardo71,Bogota,unknown,NaN,...,520,0,0,11497,False,0.0,0,1,@Citytv Una avalancha de venecos y dos millone...,tradicionales
31113,2021-03-08 00:56:15.0,@PublimetroCol Estas son las “ideas” nefastas ...,twitter.com,negative,NaN,individual,DiegoAl50239232,NaN,unknown,#alvarouribevelez,...,177,0,0,5245,False,0.0,0,1,@PublimetroCol Estas son las “ideas” nefastas ...,tradicionales


In [95]:
dups_shape = datos.pivot_table(index=['Thread_Author'], aggfunc='size')
dups_shape = pd.DataFrame(dups_shape)
mapping = {dups_shape.columns[0]:'Total_replies'}
dups_shape = dups_shape.rename(columns=mapping)
dups_shape

,Total_replies
Thread_Author,
1anoticiass,4
23Bogota,2
45SegundosCom,21
7NNoticias,2
ActivistaBogota,2
...,...
rodrigoprensa,7
seguridadmed,3
sergio_fajardo,61


In [72]:
prueba = datos[datos['Thread_Author'] == 'NoticiasCaracol']
prueba

,Date,Snippet,Domain,Sentiment,City_Code,Account_Type,Author,City,Gender,Hashtags,...,Twitter_Following,Twitter_Reply_Count,Twitter_Retweets,Twitter_Tweets,Twitter_Verified,Reach_(new),xenofobia_e_Integración_-_Integración,xenofobia_e_Integración_-_Xenofobia,Clean_text,Tipo_de_autor
1,2020-01-08 23:55:42.0,@NoticiasCaracol Los venezolanos siempre esper...,twitter.com,negative,COL.Antioquia.Medellin,individual,0rwell07,Medellin,male,NaN,...,287,0,0,13374,False,0.0,0,0,@NoticiasCaracol Los venezolanos siempre esper...,tradicionales
4,2020-01-08 20:36:22.0,"@NoticiasCaracol Veneco hijo de puta, merece l...",twitter.com,negative,COL.Bogota.Bogota,individual,carlomagno2704,Bogota,male,NaN,...,4997,0,0,36459,False,3774.0,0,1,"@NoticiasCaracol Veneco hijo de puta, merece l...",tradicionales
5,2020-01-08 20:32:05.0,@NoticiasCaracol Nada. Partiendo de que es ciu...,twitter.com,negative,COL.Bogota.Bogota,individual,avendano89,Bogota,male,NaN,...,80,0,0,3156,False,0.0,0,0,@NoticiasCaracol Nada. Partiendo de que es ciu...,tradicionales
7,2020-01-08 19:54:09.0,@NoticiasCaracol LO DE SIEMPRE CON LOS VENEZOL...,twitter.com,neutral,COL.Bogota.Bogota,individual,lpacho55,Bogota,male,NaN,...,4690,0,0,28058,False,468.0,0,0,@NoticiasCaracol LO DE SIEMPRE CON LOS VENEZOL...,tradicionales
8,2020-01-08 19:52:26.0,@NoticiasCaracol No he podido conocer al prime...,twitter.com,positive,COL.Magdalena.Santa Marta,individual,juandavid181006,Santa Marta,male,NaN,...,53,0,0,10638,False,0.0,1,0,@NoticiasCaracol No he podido conocer al prime...,tradicionales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31057,2021-03-09 17:04:41.0,@NoticiasCaracol Y el gobierno ayuda con el es...,twitter.com,neutral,COL.Bogota.Bogota,individual,Adrianarincon07,Bogota,female,NaN,...,65,0,0,530,False,0.0,0,0,@NoticiasCaracol Y el gobierno ayuda con el es...,tradicionales
31070,2021-03-09 12:44:55.0,@NoticiasCaracol Tenemos a todos los venecos d...,twitter.com,negative,COL.Bogota.Bogota,individual,bogotano_1538,Bogota,male,NaN,...,240,0,0,1564,False,0.0,0,1,@NoticiasCaracol Tenemos a todos los venecos d...,tradicionales
31077,2021-03-09 04:20:12.0,@NoticiasCaracol Nos quejamos porque nos trata...,twitter.com,negative,COL.Bogota.Bogota,individual,CarlosA51856874,Bogota,male,NaN,...,232,0,0,5487,False,0.0,1,0,@NoticiasCaracol Nos quejamos porque nos trata...,tradicionales
31082,2021-03-09 02:12:31.0,"Preocupación en las toldas uribistas, menos ve...",twitter.com,negative,COL.Bogota.Bogota,individual,RobertoMTico,Bogota,unknown,NaN,...,12480,2,14,30964,False,17709.0,0,0,"Preocupación en las toldas uribistas, menos ve...",tradicionales


In [73]:
print("Noticias Caracol generó " + str(prueba['xenofobia_e_Integración_-_Xenofobia'].sum()) + " replies xenofobos de las " + str(prueba.shape[0]) + " replies capturadas")

Noticias Caracol generó 974 replies xenofobos de las 2877 replies capturadas


In [75]:
grouped = datos.groupby('Thread_Author')
grouped.agg([np.sum]).loc[['NoticiasCaracol']]

,Date,Snippet,Domain,Sentiment,Account_Type,Author,Impact,Impressions,Thread_Entry_Type,Total_Monthly_Visitors,...,Twitter_Following,Twitter_Reply_Count,Twitter_Retweets,Twitter_Tweets,Twitter_Verified,Reach_(new),xenofobia_e_Integración_-_Integración,xenofobia_e_Integración_-_Xenofobia,Clean_text,Tipo_de_autor
,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum,...,sum,sum,sum,sum,sum,sum,sum,sum,sum,sum
Thread_Author,,,,,,,,,,,,,,,,,,,,,
NoticiasCaracol,2020-01-08 23:55:42.02020-01-08 20:36:22.02020...,@NoticiasCaracol Los venezolanos siempre esper...,twitter.comtwitter.comtwitter.comtwitter.comtw...,negativenegativenegativeneutralpositiveneutral...,individualindividualindividualindividualindivi...,0rwell07carlomagno2704avendano89lpacho55juanda...,14166.0,3568879,replyreplyreplyreplyreplyreplyreplyreplyreplyr...,17262000000000,...,1625902,772,672,32334549,3,1508895.0,224,974,@NoticiasCaracol Los venezolanos siempre esper...,tradicionalestradicionalestradicionalestradici...


In [80]:
grouped.agg([np.sum]).loc[['NoticiasCaracol']]['xenofobia_e_Integración_-_Xenofobia']

Thread_Author
NoticiasCaracol    974
Name: sum, dtype: int64

In [96]:
frame_grouped = grouped.agg([np.sum])
datos['replies_xenofobos_Thread_Author'] = ''
datos['replies_integracion_Thread_Author'] = ''
lista_autores_base = frame_grouped.index.unique()
print(lista_autores_base)
for autor in lista_autores_base:
    datos.loc[datos.Thread_Author == autor, 'replies_xenofobos_Thread_Author'] = int(frame_grouped.loc[[autor]]['xenofobia_e_Integración_-_Xenofobia']['sum'])
    datos.loc[datos.Thread_Author == autor, 'replies_integracion_Thread_Author'] = int(frame_grouped.loc[[autor]]['xenofobia_e_Integración_-_Integración']['sum'])
datos = pd.merge(datos,dups_shape,how='left',left_on='Thread_Author',right_on=dups_shape.index)
datos

Index(['1anoticiass', '23Bogota', '45SegundosCom', '7NNoticias',
       'ActivistaBogota', 'ActivoBogota21', 'Agencia_API', 'AjiDulceBogota',
       'AlPuntoCol', 'AlcaldiadeMed',
       ...
       'petrogustavo', 'primeronoticiaa', 'pulzo', 'pvirtualcom',
       'radiosantafentc', 'rodrigoprensa', 'seguridadmed', 'sergio_fajardo',
       'vivirpoblado', 'zonacero'],
      dtype='object', name='Thread_Author', length=228)


,Date,Snippet,Domain,Sentiment,City_Code,Account_Type,Author,City,Gender,Hashtags,...,Twitter_Tweets,Twitter_Verified,Reach_(new),xenofobia_e_Integración_-_Integración,xenofobia_e_Integración_-_Xenofobia,Clean_text,Tipo_de_autor,replies_xenofobos_Thread_Author,replies_integracion_Thread_Author,Total_replies
0,2020-01-08 23:58:10.0,@NTN24ve EL QUE NO SABE CON QUIEN SE METE ES L...,twitter.com,positive,COL.Antioquia.Medellin,individual,LeilaLozano3,Medellin,female,NaN,...,878,False,0.0,0,1,@NTN24ve EL QUE NO SABE CON QUIEN SE METE ES L...,,46,28,293
1,2020-01-08 23:55:42.0,@NoticiasCaracol Los venezolanos siempre esper...,twitter.com,negative,COL.Antioquia.Medellin,individual,0rwell07,Medellin,male,NaN,...,13374,False,0.0,0,0,@NoticiasCaracol Los venezolanos siempre esper...,tradicionales,974,224,2877
2,2020-01-08 23:32:58.0,@petrogustavo Yo me imagino una economía en la...,twitter.com,negative,NaN,individual,MrSwanMods,NaN,unknown,NaN,...,1216,False,0.0,0,0,@petrogustavo Yo me imagino una economía en la...,figuras,674,220,2392
3,2020-01-08 22:51:46.0,"@IvanDuque Menos hablabla y más acción, ahora ...",twitter.com,negative,COL.Bogota.Bogota,individual,billyjohnns,Bogota,male,NaN,...,115,False,0.0,0,1,"@IvanDuque Menos hablabla y más acción, ahora ...",figuras,357,240,1906
4,2020-01-08 20:36:22.0,"@NoticiasCaracol Veneco hijo de puta, merece l...",twitter.com,negative,COL.Bogota.Bogota,individual,carlomagno2704,Bogota,male,NaN,...,36459,False,3774.0,0,1,"@NoticiasCaracol Veneco hijo de puta, merece l...",tradicionales,974,224,2877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31110,2021-03-08 12:07:38.0,@petrogustavo Así eres tú un aprovechado popul...,twitter.com,negative,COL.Bogota.Bogota,individual,andirup,Bogota,male,NaN,...,352,False,0.0,1,0,@petrogustavo Así eres tú un aprovechado popul...,figuras,674,220,2392
31111,2021-03-08 12:04:41.0,@NTN24ve Y las Vacunas para los Venezolanos? Q...,twitter.com,negative,COL.Atlantico.Barranquilla,individual,MAPY2,Barranquilla,female,NaN,...,14770,False,0.0,0,0,@NTN24ve Y las Vacunas para los Venezolanos? Q...,,46,28,293
31112,2021-03-08 04:20:21.0,@Citytv Una avalancha de venecos y dos millone...,twitter.com,neutral,COL.Bogota.Bogota,individual,alexfajardo71,Bogota,unknown,NaN,...,11497,False,0.0,0,1,@Citytv Una avalancha de venecos y dos millone...,tradicionales,668,126,1663
31113,2021-03-08 00:56:15.0,@PublimetroCol Estas son las “ideas” nefastas ...,twitter.com,negative,NaN,individual,DiegoAl50239232,NaN,unknown,#alvarouribevelez,...,5245,False,0.0,0,1,@PublimetroCol Estas son las “ideas” nefastas ...,tradicionales,27,6,96


In [97]:
datos.to_excel('datos_replies.xlsx')